# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
part_1_file = "../WeatherPy/output_data/cities.csv"
# cities_df = pd.read_csv(part_1_file, index_col="City")
cities_df = pd.read_csv(part_1_file)
cities_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port elizabeth,-33.9180,25.5701,44.89,81,0,4.61,ZA,1627791686
1,1,salalah,17.0151,54.0924,77.09,100,90,5.75,OM,1627791687
2,2,arraial do cabo,-22.9661,-42.0278,66.40,61,100,13.44,BR,1627791687
3,3,ushuaia,-54.8000,-68.3000,46.06,49,40,25.32,AR,1627791687
4,4,mount pleasant,32.7941,-79.8626,84.97,93,40,5.01,US,1627791688
...,...,...,...,...,...,...,...,...,...,...
569,569,san pedro,14.3500,121.0167,88.83,69,40,8.05,PH,1627792044
570,570,savannah bight,16.4500,-85.8500,82.99,81,96,19.15,HN,1627792044
571,571,half moon bay,37.4636,-122.4286,71.42,84,1,8.05,US,1627792045
572,572,vicksburg,32.3526,-90.8779,83.93,89,1,0.00,US,1627791917


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Google developer API key
from api_keys import g_key

# Access maps with unique API key
gmaps.configure(api_key = g_key)

In [6]:
locations  = cities_df[["Lat", "Lng"]]
weights    = cities_df["Humidity"]
city_names = cities_df["City"]

# Customize the size of the figure
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
# fig=gmaps.figure()
fig = gmaps.figure(layout=figure_layout, zoom_level=2, center=(0,0))

# Plot Heatmap
heatmap = gmaps.heatmap_layer(locations, weights=weights)
fig.add_layer(heatmap)
# symbols = gmaps.symbol_layer(locations,hover_text=city_names)
# fig.add_layer(symbols)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
vac_maxtemp = (cities_df["Max Temp"] >= 70) & (cities_df["Max Temp"] < 80)
vac_windspeed = (cities_df["Wind Speed"] < 10)
vac_cloudy = (cities_df["Cloudiness"] == 0)
vac_humidity = (cities_df["Humidity"] < 80)
vac_filter = vac_maxtemp & vac_windspeed & vac_cloudy & vac_humidity
hotel_df = cities_df.loc[vac_filter].dropna()
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
197,197,erzin,36.9559,36.2000,79.52,70,0,0.78,TR,1627791791
241,241,novikovo,51.1537,37.8879,73.74,53,0,2.44,RU,1627791815
247,247,fasa,28.9383,53.6482,78.04,34,0,0.00,IR,1627791819
272,272,farah,32.5000,62.5000,79.95,14,0,7.27,AF,1627791833
286,286,samsun,41.2500,36.3333,75.99,64,0,2.30,TR,1627791840
320,320,yulara,-25.2406,130.9889,71.44,35,0,6.91,AU,1627791857
426,426,turayf,31.6725,38.6637,73.65,43,0,6.91,SA,1627791903
467,467,ust-maya,60.4167,134.5333,79.75,32,0,4.43,RU,1627791998
471,471,susurluk,39.9136,28.1578,78.58,60,0,1.30,TR,1627792000
537,537,akcakoca,41.0866,31.1162,73.92,54,0,3.06,TR,1627792030


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df["Hotel Name"] = ""
my_radius = 5000

for idx in hotel_df.index:
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    mylat = hotel_df.loc[idx, "Lat"]
    mylng = hotel_df.loc[idx, "Lng"]
    location = f"{mylat}, {mylng}"
    params = {
        "location": location,
        "key": g_key,
        "type": "lodging",
        "keyword": "hotel",
        "radius": my_radius
    }
    response = requests.get(base_url, params).json()
    print(f"Working on index {idx}")
    if response["status"] == 'ZERO_RESULTS':
        print(f"Zero results for index {idx}. Skipping...")
        hotel_name = f"None found in radius: {my_radius}"
    else:
        hotel_name = response["results"][0]["name"]
    hotel_df.loc[idx, "Hotel Name"] = hotel_name
hotel_df

Working on index 197
Working on index 241
Zero results for index 241. Skipping...
Working on index 247
Working on index 272
Zero results for index 272. Skipping...
Working on index 286
Working on index 320
Working on index 426
Working on index 467
Zero results for index 467. Skipping...
Working on index 471
Working on index 537


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
197,197,erzin,36.9559,36.2000,79.52,70,0,0.78,TR,1627791791,Hattusa Vacation Thermal Club Erzin
241,241,novikovo,51.1537,37.8879,73.74,53,0,2.44,RU,1627791815,None found in radius: 5000
247,247,fasa,28.9383,53.6482,78.04,34,0,0.00,IR,1627791819,Jahangardi Hotel
272,272,farah,32.5000,62.5000,79.95,14,0,7.27,AF,1627791833,None found in radius: 5000
286,286,samsun,41.2500,36.3333,75.99,64,0,2.30,TR,1627791840,Anemon Samsun Otel
320,320,yulara,-25.2406,130.9889,71.44,35,0,6.91,AU,1627791857,Sails in the Desert
426,426,turayf,31.6725,38.6637,73.65,43,0,6.91,SA,1627791903,Golden Dune Hotel
467,467,ust-maya,60.4167,134.5333,79.75,32,0,4.43,RU,1627791998,None found in radius: 5000
471,471,susurluk,39.9136,28.1578,78.58,60,0,1.30,TR,1627792000,Grand Rimedya Hotel Susurluk
537,537,akcakoca,41.0866,31.1162,73.92,54,0,3.06,TR,1627792030,Akcakoca Poyraz Hotel - Apartments


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map

# Display figure
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…